CARGAR ARCHIVO DE MEDICION


In [29]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

# Creación de ventana 
root = tk.Tk()
root.withdraw()

# Abrir cuadro de diálogo para seleccionar el archivo de medición
ruta_archivo = filedialog.askopenfilename(
    title="Selecciona un archivo Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

# Verificar si el usuario seleccionó un archivo
if ruta_archivo:
    # Leer todas las hojas del Excel
    excel_data = pd.read_excel(ruta_archivo, sheet_name=None, engine='openpyxl')

    # Mostrar nombres de hojas disponibles
    print("Hojas disponibles:", list(excel_data.keys()))

    # Acceder a la primera hoja
    hoja_resumen = list(excel_data.values())[0]

    # Seleccionar solo las dos primeras columnas
    df_resumen = hoja_resumen.iloc[:, :2]

    # Mostrar resultados
    print("\nResumen (dos primeras columnas):")
    print(df_resumen)
else:
    print("No se seleccionó ningún archivo.")

Hojas disponibles: ['Resumen', 'Historia']

Resumen (dos primeras columnas):
      SPLAS   67.7
0     SPLBS     72
1     SPLCS   76.3
2     SPLZS   77.4
3     6.3Hz      0
4       8Hz      0
5      10Hz      0
6    12.5Hz      0
7      16Hz   5.57
8      20Hz  13.69
9      25Hz  18.79
10   31.5Hz   25.3
11     40Hz  30.04
12     50Hz  38.53
13     63Hz  43.91
14     80Hz  41.27
15    100Hz  45.92
16    125Hz  50.37
17    160Hz  54.27
18    200Hz   51.4
19    250Hz  52.48
20    315Hz  54.79
21    400Hz  54.28
22    500Hz  55.17
23    630Hz  56.27
24    800Hz  57.72
25     1kHz  59.09
26  1.25kHz  58.66
27   1.6kHz   57.3
28     2kHz  55.93
29   2.5kHz  55.26
30  3.15kHz   55.2
31     4kHz  53.65
32     5kHz  49.42
33   6.3kHz   44.7
34     8kHz     40
35    10kHz  34.59
36  12.5kHz  27.02
37    16kHz  17.06
38    20kHz   2.57
39     LAeq      0
40     LAeq   67.7
41    LAIeq   69.8
42      L90   60.3
43   LASmax   78.7
44   LASmin   57.9
45      LAS   62.9
46     LAeq   67.7
47      LAF

HISTOGRAMA DEL ARCHIVO DE MEDICION

In [30]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, RadioButtonGroup, CustomJS
from bokeh.layouts import column

output_notebook()


# Asumimos que ya cargaste el Excel y lo tienes en "hoja_resumen"
hoja_resumen.columns = hoja_resumen.columns.str.strip()
col_descriptor = hoja_resumen.columns[0]
col_valor = hoja_resumen.columns[1]

# Filtrar solo las filas que contienen frecuencias entre 6.3Hz y 20kHz
# Lista de etiquetas de frecuencia (bandas de tercio de octava)
bandas_tercio = [
    "6.3Hz", "8Hz", "10Hz", "12.5Hz", "16Hz", "20Hz", "25Hz", "31.5Hz",
    "40Hz", "50Hz", "63Hz", "80Hz", "100Hz", "125Hz", "160Hz", "200Hz",
    "250Hz", "315Hz", "400Hz", "500Hz", "630Hz", "800Hz", "1kHz", "1.25kHz",
    "1.6kHz", "2kHz", "2.5kHz", "3.15kHz", "4kHz", "5kHz", "6.3kHz", "8kHz",
    "10kHz", "12.5kHz", "16kHz", "20kHz"
]

# Filtrar filas cuyo descriptor esté en la lista de bandas
df_frecuencias = hoja_resumen[hoja_resumen[col_descriptor].isin(bandas_tercio)].copy()

# Asegurar que los valores sean numéricos
df_frecuencias[col_valor] = pd.to_numeric(df_frecuencias[col_valor], errors='coerce')

# ----------------------------------------
# 1. INGRESO DE PONDERACIÓN ORIGINAL
# ----------------------------------------
ponderacion_original = input("¿En qué ponderación están originalmente los datos? (Z, A, C): ").strip().upper()

if ponderacion_original not in ["Z", "A", "C"]:
    raise ValueError("Entrada inválida. Debe ser 'Z', 'A' o 'C'.")

# ----------------------------------------
# 2. PREPARAR LOS DATOS
# ----------------------------------------
df_frecuencias.columns = df_frecuencias.columns.str.strip()
col_descriptor = df_frecuencias.columns[0]
col_valor = df_frecuencias.columns[1]

frecs = df_frecuencias[col_descriptor].values
valores_ingresados = df_frecuencias[col_valor].values

# ----------------------------------------
# 3. TABLA DE PONDERACIONES (ya corregida)
# ----------------------------------------
df_ponderacion = pd.DataFrame({
    'Frecuencia': [
        "6.3Hz", "8Hz", "10Hz", "12.5Hz", "16Hz", "20Hz", "25Hz", "31.5Hz",
        "40Hz", "50Hz", "63Hz", "80Hz", "100Hz", "125Hz", "160Hz", "200Hz",
        "250Hz", "315Hz", "400Hz", "500Hz", "630Hz", "800Hz", "1kHz", "1.25kHz",
        "1.6kHz", "2kHz", "2.5kHz", "3.15kHz", "4kHz", "5kHz", "6.3kHz", "8kHz",
        "10kHz", "12.5kHz", "16kHz", "20kHz"
    ],
    'A': [
        -85.4, -77.8, -70.4, -63.4, -56.7, -50.5, -44.7, -39.4, -34.6, -30.2, -26.2,
        -22.5, -19.1, -16.1, -13.4, -10.9, -8.6, -6.6, -4.8, -3.2, -1.9,
        -0.8, 0, 0.6, 1.0, 1.2, 1.3, 1.2, 1.0, 0.5,
        -0.1, -1.1, -2.5, -4.3, -6.6, -9.3
    ],
    'C': [
        -21.3, -17.7, -14.3, -11.2, -8.5, -6.2, -4.4, -3, -2, -1.3, -0.8,
        -0.5, -0.3, -0.2, -0.1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, -0.1, -0.2, -0.3, -0.5, -0.8, -1.3,
        -2, -3, -4.4, -6.2, -8.5, -11.2
    ]
})

# ----------------------------------------
# 4. CALCULAR VALORES SEGÚN PONDERACIÓN ORIGINAL
# ----------------------------------------
df_pond_indexed = df_ponderacion.set_index('Frecuencia')
correccion_A = df_pond_indexed.loc[frecs, 'A'].values
correccion_C = df_pond_indexed.loc[frecs, 'C'].values

if ponderacion_original == "Z":
    valores_Z = valores_ingresados
elif ponderacion_original == "A":
    valores_Z = valores_ingresados - correccion_A
elif ponderacion_original == "C":
    valores_Z = valores_ingresados - correccion_C

valores_A = valores_Z + correccion_A
valores_C = valores_Z + correccion_C

def calcular_Lp_total(valores):
    return 10 * np.log10(np.sum(10 ** (valores / 10)))

Lp_total_Z = calcular_Lp_total(valores_Z)
Lp_total_A = calcular_Lp_total(valores_A)
Lp_total_C = calcular_Lp_total(valores_C)

frecuencias_ext = list(frecs) + ['TOTAL']
valores_Z_ext = list(np.round(valores_Z, 2)) + [round(Lp_total_Z, 2)]
valores_A_ext = list(np.round(valores_A, 2)) + [round(Lp_total_A, 2)]
valores_C_ext = list(np.round(valores_C, 2)) + [round(Lp_total_C, 2)]

colores_Z = ["firebrick"] * len(frecuencias_ext)
colores_A = ["seagreen"] * len(frecuencias_ext)
colores_C = ["royalblue"] * len(frecuencias_ext)

# Conversión para JavaScript (listas nativas)
valores_Z_ext = list(map(float, valores_Z_ext))
valores_A_ext = list(map(float, valores_A_ext))
valores_C_ext = list(map(float, valores_C_ext))

colores_Z = list(map(str, colores_Z))
colores_A = list(map(str, colores_A))
colores_C = list(map(str, colores_C))

# ----------------------------------------
# 5. GRAFICAR EN BOKEH
# ----------------------------------------
source = ColumnDataSource(data={
    'frecuencia': frecuencias_ext,
    'valor': valores_Z_ext,
    'color': colores_Z
})

p = figure(
    x_range=frecuencias_ext,
    height=400,
    width=950,
    title="Histograma interactivo con ponderación (Z, A, C) y Lp total",
    toolbar_location=None
)

p.vbar(
    x='frecuencia',
    top='valor',
    width=0.8,
    source=source,
    fill_color='color',
    line_color="black"
)

hover = HoverTool(tooltips=[
    ("Frecuencia", "@frecuencia"),
    ("Nivel", "@valor dB")
])
p.add_tools(hover)

p.xaxis.major_label_orientation = 0.75
p.xaxis.axis_label = "Frecuencia [Hz] / TOTAL"
p.yaxis.axis_label = "Nivel de presión sonora [dB]"

# ----------------------------------------
# 6. BOTÓN DE PONDERACIÓN INTERACTIVO
# ----------------------------------------
botones = RadioButtonGroup(labels=["Z (sin)", "A", "C"], active=0)

botones.js_on_change("active", CustomJS(args=dict(source=source), code="""
    const datos = source.data;
    const modo = cb_obj.active;

    const valores_Z = """ + str(valores_Z_ext) + """;
    const valores_A = """ + str(valores_A_ext) + """;
    const valores_C = """ + str(valores_C_ext) + """;

    const colores_Z = """ + str(colores_Z) + """;
    const colores_A = """ + str(colores_A) + """;
    const colores_C = """ + str(colores_C) + """;

    let nuevos_valores = [];
    let nuevos_colores = [];

    if (modo === 0) {
        nuevos_valores = valores_Z;
        nuevos_colores = colores_Z;
    } else if (modo === 1) {
        nuevos_valores = valores_A;
        nuevos_colores = colores_A;
    } else if (modo === 2) {
        nuevos_valores = valores_C;
        nuevos_colores = colores_C;
    }

    for (let i = 0; i < datos['valor'].length; i++) {
        datos['valor'][i] = nuevos_valores[i];
        datos['color'][i] = nuevos_colores[i];
    }

    source.change.emit();
"""))

# ----------------------------------------
# 7. MOSTRAR GRÁFICO
# ----------------------------------------
show(column(botones, p))

Loading BokehJS ...

TIME HISTORY ARCHIVO DE MEDICION

In [3]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.layouts import column

output_notebook()

# ---------------------------
# 1. Extraer datos de 'Historia'
# ---------------------------
df_historia = excel_data['Historia']

# Limpiar y normalizar columnas
df_historia.columns = df_historia.columns.str.strip()
col_tiempo = df_historia.columns[0]
col_nivel = df_historia.columns[1]

# Convertir tiempos y niveles
df_historia[col_tiempo] = pd.to_datetime(df_historia[col_tiempo], errors='coerce')
df_historia[col_nivel] = pd.to_numeric(df_historia[col_nivel], errors='coerce')
df_historia = df_historia.dropna(subset=[col_tiempo, col_nivel])

# ---------------------------
# 2. Calcular LAeq
# ---------------------------
niveles = df_historia[col_nivel].values
t_i = 60  # segundos por muestra (1 minuto)
numerador = np.sum(t_i * 10 ** (niveles / 10))
denominador = len(niveles) * t_i
laeq = 10 * np.log10(numerador / denominador)
laeq = round(laeq, 2)

# ---------------------------
# 3. Graficar niveles y LAeq
# ---------------------------
source = ColumnDataSource(data={
    'tiempo': df_historia[col_tiempo],
    'nivel': niveles
})

p = figure(
    x_axis_type='datetime',
    height=400,
    width=950,
    title=f"Niveles sonoros por minuto - LAeq total = {laeq} dB",
    toolbar_location=None
)

p.line(x='tiempo', y='nivel', source=source, line_width=2, color="darkorange", legend_label="Nivel dB(A)")
p.circle(x='tiempo', y='nivel', source=source, size=5, color="darkorange")

# Línea horizontal con el LAeq
laeq_line = Span(location=laeq, dimension='width', line_color='blue', line_dash='dashed', line_width=2)
p.add_layout(laeq_line)

# Tooltip
hover = HoverTool(tooltips=[
    ("Hora", "@tiempo{%H:%M:%S}"),
    ("Nivel", "@nivel dB")
], formatters={'@tiempo': 'datetime'})
p.add_tools(hover)

p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel de presión sonora [dB]"
p.legend.location = "top_left"

# ---------------------------
# 4. Mostrar gráfico
# ---------------------------
show(p)


Loading BokehJS ...

SUMMARY HISTOGRAMA DE TODOS LOS ARCHIVOS

In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, RadioButtonGroup, CustomJS
from bokeh.layouts import column

# -------------------------------------
# 1. SELECCIÓN DE ARCHIVOS
# -------------------------------------
root = tk.Tk()
root.withdraw()

rutas_archivos = filedialog.askopenfilenames(
    title="Selecciona los archivos Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

# -------------------------------------
# 2. LISTA DE FRECUENCIAS Y PONDERACIONES
# -------------------------------------
bandas_tercio = [
    "6.3Hz", "8Hz", "10Hz", "12.5Hz", "16Hz", "20Hz", "25Hz", "31.5Hz",
    "40Hz", "50Hz", "63Hz", "80Hz", "100Hz", "125Hz", "160Hz", "200Hz",
    "250Hz", "315Hz", "400Hz", "500Hz", "630Hz", "800Hz", "1kHz", "1.25kHz",
    "1.6kHz", "2kHz", "2.5kHz", "3.15kHz", "4kHz", "5kHz", "6.3kHz", "8kHz",
    "10kHz", "12.5kHz", "16kHz", "20kHz"
]

ponderaciones = pd.DataFrame({
    "Frecuencia": bandas_tercio,
    "A": [
        -85.4, -77.8, -70.4, -63.4, -56.7, -50.5, -44.7, -39.4, -34.6, -30.2, -26.2,
        -22.5, -19.1, -16.1, -13.4, -10.9, -8.6, -6.6, -4.8, -3.2, -1.9,
        -0.8, 0, 0.6, 1.0, 1.2, 1.3, 1.2, 1.0, 0.5,
        -0.1, -1.1, -2.5, -4.3, -6.6, -9.3
    ],
    "C": [
        -21.3, -17.7, -14.3, -11.2, -8.5, -6.2, -4.4, -3, -2, -1.3, -0.8,
        -0.5, -0.3, -0.2, -0.1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, -0.1, -0.2, -0.3, -0.5, -0.8, -1.3,
        -2, -3, -4.4, -6.2, -8.5, -11.2
    ]
}).set_index("Frecuencia")

# -------------------------------------
# 3. FUNCIONES AUXILIARES
# -------------------------------------
def suma_energetica(niveles_db, promedio=True):
    lineales = 10 ** (np.array(niveles_db) / 10)
    suma = np.sum(lineales)
    if promedio:
        return round(10 * np.log10((1/5) * suma), 3)
    else:
        return round(10 * np.log10(suma), 3)

acumulador = {banda: [] for banda in bandas_tercio}

# -------------------------------------
# 4. PROCESAR ARCHIVOS
# -------------------------------------
for ruta in rutas_archivos:
    try:
        nombre = ruta.split("/")[-1] if "/" in ruta else ruta.split("\\")[-1]
        entrada = ""
        while entrada not in ["A", "C", "Z"]:
            entrada = input(f"¿Ponderación original del archivo '{nombre}'? (Z, A, C): ").strip().upper()

        excel_data = pd.read_excel(ruta, sheet_name=None, engine='openpyxl')
        hoja_resumen = list(excel_data.values())[0]
        hoja_resumen.columns = hoja_resumen.columns.str.strip()
        col_descriptor = hoja_resumen.columns[0]
        col_valor = hoja_resumen.columns[1]

        df_bandas = hoja_resumen[hoja_resumen[col_descriptor].isin(bandas_tercio)].copy()
        df_bandas[col_valor] = pd.to_numeric(df_bandas[col_valor], errors='coerce')
        df_bandas = df_bandas.dropna(subset=[col_valor])
        df_bandas.set_index(col_descriptor, inplace=True)

        for banda in bandas_tercio:
            if banda in df_bandas.index:
                valor = df_bandas.loc[banda, col_valor]
                if entrada == "A":
                    valor_z = valor - ponderaciones.loc[banda, "A"]
                elif entrada == "C":
                    valor_z = valor - ponderaciones.loc[banda, "C"]
                else:
                    valor_z = valor
                acumulador[banda].append(valor_z)

    except Exception as e:
        print(f"Error al procesar {ruta}: {e}")

# -------------------------------------
# 5. CALCULOS
# -------------------------------------
niveles_Z = {banda: suma_energetica(acumulador[banda]) for banda in bandas_tercio}
df_resultado = pd.DataFrame.from_dict(niveles_Z, orient='index', columns=['Z']).reset_index()
df_resultado = df_resultado.rename(columns={'index': 'Frecuencia'})
df_resultado['A'] = df_resultado['Z'] + ponderaciones['A'].values
df_resultado['C'] = df_resultado['Z'] + ponderaciones['C'].values

# Cálculo correcto sin promedio para Lp totales
Lp_total_Z = suma_energetica(df_resultado['Z'], promedio=False)
Lp_total_A = suma_energetica(df_resultado['A'], promedio=False)
Lp_total_C = suma_energetica(df_resultado['C'], promedio=False)

df_resultado.loc[len(df_resultado)] = ['TOTAL', Lp_total_Z, Lp_total_A, Lp_total_C]

# -------------------------------------
# 6. GRAFICAR CON BOKEH
# -------------------------------------
frecuencias_ext = df_resultado["Frecuencia"].tolist()
valores_Z_ext = df_resultado["Z"].tolist()
valores_A_ext = df_resultado["A"].tolist()
valores_C_ext = df_resultado["C"].tolist()

colores_Z = ["firebrick"] * len(frecuencias_ext)
colores_A = ["seagreen"] * len(frecuencias_ext)
colores_C = ["royalblue"] * len(frecuencias_ext)

source = ColumnDataSource(data={
    'frecuencia': frecuencias_ext,
    'valor': valores_Z_ext,
    'color': colores_Z
})

p = figure(
    x_range=frecuencias_ext,
    height=400,
    width=950,
    title="Histograma interactivo con ponderación (Z, A, C) y Lp total",
    toolbar_location=None
)

p.vbar(
    x='frecuencia',
    top='valor',
    width=0.8,
    source=source,
    fill_color='color',
    line_color="black"
)

hover = HoverTool(tooltips=[
    ("Frecuencia", "@frecuencia"),
    ("Nivel", "@valor dB")
])
p.add_tools(hover)

p.xaxis.major_label_orientation = 0.75
p.xaxis.axis_label = "Frecuencia [Hz] / TOTAL"
p.yaxis.axis_label = "Nivel de presión sonora [dB]"

botones = RadioButtonGroup(labels=["Z (sin)", "A", "C"], active=0)

botones.js_on_change("active", CustomJS(args=dict(source=source), code=f"""
    const datos = source.data;
    const modo = cb_obj.active;

    const valores_Z = {valores_Z_ext};
    const valores_A = {valores_A_ext};
    const valores_C = {valores_C_ext};

    const colores_Z = {colores_Z};
    const colores_A = {colores_A};
    const colores_C = {colores_C};

    let nuevos_valores = [];
    let nuevos_colores = [];

    if (modo === 0) {{
        nuevos_valores = valores_Z;
        nuevos_colores = colores_Z;
    }} else if (modo === 1) {{
        nuevos_valores = valores_A;
        nuevos_colores = colores_A;
    }} else if (modo === 2) {{
        nuevos_valores = valores_C;
        nuevos_colores = colores_C;
    }}

    for (let i = 0; i < datos['valor'].length; i++) {{
        datos['valor'][i] = nuevos_valores[i];
        datos['color'][i] = nuevos_colores[i];
    }}

    source.change.emit();
"""))

output_file("histograma_ponderado.html")
show(column(botones, p))


In [5]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.layouts import column

# --------------------------------------
# 1. Selección de archivos
# --------------------------------------
root = tk.Tk()
root.withdraw()

rutas_archivos = filedialog.askopenfilenames(
    title="Selecciona los archivos Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

# --------------------------------------
# 2. Procesar cada hoja 'Historia'
# --------------------------------------
todos_los_niveles = []
todos_los_tiempos = []

for ruta in rutas_archivos:
    try:
        excel_data = pd.read_excel(ruta, sheet_name="Historia", engine='openpyxl')
        excel_data.columns = excel_data.columns.str.strip()

        col_tiempo = excel_data.columns[0]
        col_nivel = excel_data.columns[1]

        df = excel_data[[col_tiempo, col_nivel]].copy()
        df[col_tiempo] = pd.to_datetime(df[col_tiempo], errors='coerce')
        df[col_nivel] = pd.to_numeric(df[col_nivel], errors='coerce')
        df = df.dropna(subset=[col_tiempo, col_nivel])

        todos_los_tiempos.extend(df[col_tiempo].tolist())
        todos_los_niveles.extend(df[col_nivel].tolist())

    except Exception as e:
        print(f"Error al procesar {ruta}: {e}")

# --------------------------------------
# 3. Crear DataFrame combinado y ordenar
# --------------------------------------
df_total = pd.DataFrame({
    "Tiempo": todos_los_tiempos,
    "Nivel": todos_los_niveles
})

df_total = df_total.sort_values("Tiempo").reset_index(drop=True)

# --------------------------------------
# 4. Calcular LAeq total
# --------------------------------------
# Cada punto es de 1 minuto => 60 segundos
t_i = 60
niveles = df_total["Nivel"].values

numerador = np.sum(t_i * 10 ** (niveles / 10))
denominador = len(niveles) * t_i
laeq_total = round(10 * np.log10(numerador / denominador), 2)

# --------------------------------------
# 5. Graficar en Bokeh
# --------------------------------------
source = ColumnDataSource(data={
    "tiempo": df_total["Tiempo"],
    "nivel": df_total["Nivel"]
})

p = figure(
    x_axis_type='datetime',
    height=400,
    width=950,
    title=f"Niveles sonoros por minuto (todos los archivos) - LAeq total = {laeq_total} dB",
    toolbar_location=None
)

p.line(x='tiempo', y='nivel', source=source, line_width=2, color="darkorange", legend_label="Nivel dB(A)")
p.circle(x='tiempo', y='nivel', source=source, size=5, color="darkorange")

# Línea horizontal LAeq
laeq_line = Span(location=laeq_total, dimension='width', line_color='blue', line_dash='dashed', line_width=2)
p.add_layout(laeq_line)

# HoverTool
hover = HoverTool(tooltips=[
    ("Hora", "@tiempo{%H:%M:%S}"),
    ("Nivel", "@nivel dB")
], formatters={'@tiempo': 'datetime'})

p.add_tools(hover)

p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel de presión sonora [dB]"
p.legend.location = "top_left"

# --------------------------------------
# 6. Mostrar
# --------------------------------------
output_file("grafico_LAeq_total.html")
show(p)


In [6]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Span
from bokeh.palettes import Category10
from bokeh.layouts import column
import os

# --------------------------------------
# 1. Selección de archivos
# --------------------------------------
root = tk.Tk()
root.withdraw()

rutas_archivos = filedialog.askopenfilenames(
    title="Selecciona los archivos Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

# Paleta de colores
colores = Category10[10]  # Hasta 10 colores distintos

# --------------------------------------
# 2. Procesar cada hoja 'Historia'
# --------------------------------------
data_sources = []
niveles_acumulados = []

for i, ruta in enumerate(rutas_archivos):
    try:
        nombre_archivo = os.path.basename(ruta)
        excel_data = pd.read_excel(ruta, sheet_name="Historia", engine='openpyxl')
        excel_data.columns = excel_data.columns.str.strip()

        col_tiempo = excel_data.columns[0]
        col_nivel = excel_data.columns[1]

        df = excel_data[[col_tiempo, col_nivel]].copy()
        df[col_tiempo] = pd.to_datetime(df[col_tiempo], errors='coerce')
        df[col_nivel] = pd.to_numeric(df[col_nivel], errors='coerce')
        df = df.dropna(subset=[col_tiempo, col_nivel])

        # Guardar niveles para LAeq
        niveles_acumulados.extend(df[col_nivel].tolist())

        # Preparar ColumnDataSource con color y nombre
        source = ColumnDataSource(data={
            'tiempo': df[col_tiempo],
            'nivel': df[col_nivel],
        })

        data_sources.append((source, nombre_archivo, colores[i % len(colores)]))

    except Exception as e:
        print(f"Error al procesar {ruta}: {e}")

# --------------------------------------
# 3. Calcular LAeq total
# --------------------------------------
t_i = 60
niveles = np.array(niveles_acumulados)
numerador = np.sum(t_i * 10 ** (niveles / 10))
denominador = len(niveles) * t_i
laeq_total = round(10 * np.log10(numerador / denominador), 2)

# --------------------------------------
# 4. Graficar en Bokeh
# --------------------------------------
p = figure(
    x_axis_type='datetime',
    height=450,
    width=950,
    title=f"Niveles sonoros por minuto (múltiples archivos) - LAeq total = {laeq_total} dB",
    toolbar_location=None
)

for source, nombre_archivo, color in data_sources:
    p.line(x='tiempo', y='nivel', source=source, line_width=2, color=color, legend_label=nombre_archivo)
    p.circle(x='tiempo', y='nivel', source=source, size=5, color=color)

# Línea horizontal LAeq
laeq_line = Span(location=laeq_total, dimension='width', line_color='blue', line_dash='dashed', line_width=2)
p.add_layout(laeq_line)

# Hover
hover = HoverTool(tooltips=[
    ("Hora", "@tiempo{%H:%M:%S}"),
    ("Nivel", "@nivel dB")
], formatters={'@tiempo': 'datetime'})
p.add_tools(hover)

p.xaxis.axis_label = "Hora"
p.yaxis.axis_label = "Nivel de presión sonora [dB]"
p.legend.location = "bottom_left"
p.legend.click_policy = "hide"

# --------------------------------------
# 5. Mostrar
# --------------------------------------
output_file("grafico_LAeq_por_archivo.html")
show(p)
